In [118]:
import glob

import pandas as pd

from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import torch

from tqdm import tqdm

tqdm.pandas()

In [119]:
data_base_dir = "data/by_date"

dates = []
dfs = {}
for fpath in glob.glob(data_base_dir + "/*"):
    df = pd.read_csv(fpath)
    date = df["date"].iloc[0]
    dfs[date] = df  # [df["question"].notnull()]
    dates.append(date)

dates.sort()

In [120]:
dfs[dates[0]]

,speaker,text,type,election_year,date,candidate,qud,question,similarity_cls,similarity_max,similarity_mean,similarity_sbert,party,question_clean
0,Howard Smith,Good evening. The television and radio station...,Pres,1960,1960-09-26,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
1,John Kennedy,"Mr. Smith, Mr. Nixon. In the election of 1860,...",Pres,1960,1960-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
2,Howard Smith,And now the opening statement by Vice Presiden...,Pres,1960,1960-09-26,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
3,Richard Nixon,"Mr. Smith, Senator Kennedy. The things that Se...",Pres,1960,1960-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,Republican,NaN
4,Howard Smith,"Thank you, Mr. Nixon. That completes the openi...",Pres,1960,1960-09-26,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Howard Smith,Three minutes and twenty seconds for each cand...,Pres,1960,1960-09-26,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
64,Richard Nixon,"Thank you, Mr. Smith. Senator Kennedy. First o...",Pres,1960,1960-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,Republican,NaN
65,Howard Smith,"Senator Kennedy, your conclusion.",Pres,1960,1960-09-26,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
66,John Kennedy,The point was made by Mr. Nixon that the Sovie...,Pres,1960,1960-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN


In [121]:
dfs[dates[0]][dfs[dates[0]]["question"].notnull()]

,speaker,text,type,election_year,date,candidate,qud,question,similarity_cls,similarity_max,similarity_mean,similarity_sbert,party,question_clean
6,John Kennedy,"Well, the Vice President and I came to the Con...",Pres,1960,1960-09-26,1,Which political party's policies and leadershi...,"Senator, the Vice President in his campaign ha...",0.955318,0.945188,0.880888,0.715020,Democratic,Which point of view and which party do we want...
11,Richard Nixon,It would be rather difficult to cover them in ...,Pres,1960,1960-09-26,1,What recommendations have you made following y...,"Mr. Vice President, your campaign stresses the...",0.980144,0.976831,0.956235,0.846666,Republican,What recommendations have you made after your ...
17,John Kennedy,"Well, because I think that if the federal gove...",Pres,1960,1960-09-26,1,What role should the federal government play i...,"Id like to ask this; Its a fact, I think, that...",0.919118,0.931436,0.859552,0.647737,Democratic,What agricultural policy changes do you sugges...
21,Richard Nixon,"Well, I would suggest, Mr. Vanocur, that uh – ...",Pres,1960,1960-09-26,1,What is the role of presidential advisers in d...,"Uh – Mr. Vice President, since the question of...",0.921347,0.953946,0.888321,0.606748,Republican,What is your experience in providing advice to...
26,John Kennedy,I didnt indicate. I did not advocate reducing ...,Pres,1960,1960-09-26,1,Why do you believe it is not feasible to signi...,"Senator Kennedy, in connection with these prob...",0.923712,0.943947,0.892095,0.760218,Democratic,Have you suggested that we should be able to r...
32,John Kennedy,"– reducing the interest rate. In my judgment, ...",Pres,1960,1960-09-26,1,What economic policies should be implemented t...,"Senator, I believe in – in one of your speeche...",0.925728,0.948196,0.898400,0.658894,Democratic,How do you propose to finance essential progra...
37,Richard Nixon,Im awfully glad you ge- got that question beca...,Pres,1960,1960-09-26,1,Why should the federal government avoid taking...,Mr. Vice President you mentioned schools and i...,0.955854,0.963308,0.893042,0.857709,Republican,Why should there be any question about the fed...
44,John Kennedy,"If I may take the bills, we did pass in the Se...",Pres,1960,1960-09-26,1,What challenges does Congress face in passing ...,"Senator, youve been promising the voters that ...",0.932877,0.939191,0.879735,0.563676,Democratic,What will you do to ensure support for key pro...
53,John Kennedy,"Well, I think theyre serious. I think its a ma...",Pres,1960,1960-09-26,1,What should the United States do to address in...,"Senator Kennedy, on another subject, Communism...",0.952929,0.934089,0.837785,0.719480,Democratic,What measures should we take to sustain the De...
58,Richard Nixon,Not at all. As a matter of fact your question ...,Pres,1960,1960-09-26,1,What factors should be considered when assessi...,Mr. Vice President uh – in one of your earlier...,0.952243,0.936278,0.862673,0.647500,Republican,What is the role of individual enterprise in A...


In [122]:
sum([len(df) for df in dfs.values()])

2716

In [123]:
sum([len(df[df["question"].notnull()]) for df in dfs.values()])

271

In [124]:
for i, d in enumerate(dates):
    print(f"{i}\t{d}\t{len(dfs[d])}\t{len(dfs[d].columns)}")

0	1960-09-26	68	14
1	1976-09-23	94	14
2	1980-09-21	76	14
3	1984-10-07	134	14
4	1988-09-25	161	14
5	1992-10-11	92	14
6	1996-10-06	144	14
7	2000-10-03	166	14
8	2004-09-30	142	14
9	2008-09-26	189	14
10	2012-10-03	210	14
11	2016-09-26	308	14
12	2020-09-29	932	14


In [125]:
for i, d in enumerate(dates):
    df_f = dfs[d][dfs[d]["question"].notnull()]
    print(f"{i}\t{d}\t{len(df_f)}\t{len(df_f.columns)}")

0	1960-09-26	10	14
1	1976-09-23	20	14
2	1980-09-21	12	14
3	1984-10-07	30	14
4	1988-09-25	23	14
5	1992-10-11	11	14
6	1996-10-06	21	14
7	2000-10-03	26	14
8	2004-09-30	18	14
9	2008-09-26	17	14
10	2012-10-03	25	14
11	2016-09-26	19	14
12	2020-09-29	39	14


In [126]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

In [127]:
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

In [128]:
def encode_text(text, sbert=False, pooling_method="cls"):
    if sbert:
        embedding = (
            sbert_model.encode(text, convert_to_tensor=True).cpu().reshape(1, -1)
        )
    else:
        inputs = bert_tokenizer(
            text, return_tensors="pt", truncation=True, padding=True, max_length=512
        )
        with torch.no_grad():
            outputs = bert_model(**inputs)

        # Use the [CLS] token's embedding as the sentence embedding
        if pooling_method == "cls":
            embedding = outputs.last_hidden_state[:, 0, :]
        elif pooling_method == "max":
            embedding = outputs.last_hidden_state.max(dim=1).values
        elif pooling_method == "mean":
            embedding = outputs.last_hidden_state.mean(dim=1)
    return embedding

In [129]:
def calculate_similarity(text1, text2, sbert=False, pooling_method="cls"):
    embedding1 = encode_text(text1, sbert=sbert, pooling_method=pooling_method)
    embedding2 = encode_text(text2, sbert=sbert, pooling_method=pooling_method)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

In [130]:
calculate_similarity("test1", "test2")

np.float32(0.97823465)

In [131]:
calculate_similarity("test1", "test2", pooling_method="max")

np.float32(0.9586725)

In [132]:
calculate_similarity("test1", "test2", sbert=True)

np.float32(0.8835417)

In [133]:
i = 6

In [134]:
dfs[dates[i]]

,speaker,text,type,election_year,date,candidate,qud,question,similarity_cls,similarity_max,similarity_mean,similarity_sbert,party,question_clean
0,Jim Lehrer,Good evening from the Bushnell Theatre in Hart...,Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
1,Bill Clinton,"Thank you, Jim. And thank you to the people of...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
2,Jim Lehrer,"Senator Dole, two minutes.",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
3,Bob Dole,"Thank you. Thank you, Mr. President, for those...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Republican,NaN
4,Jim Lehrer,"Mr. President, first question. There is a majo...",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Bill Clinton,I think a lot of people deserve credit and Ive...,Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
140,Jim Lehrer,"All right. Thats the last question, the last a...",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
141,Bill Clinton,"Well, first, Jim, let me thank you and thank y...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
142,Jim Lehrer,"Senator Dole, your closing statement, sir.",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN


In [135]:
dfs[dates[i]]["similarity_cls"] = dfs[dates[i]].progress_apply(
    lambda row: (
        calculate_similarity(row["qud"], row["question_clean"])
        if not pd.isna(row["question"])
        else None
    ),
    axis=1,
)
dfs[dates[i]]["similarity_max"] = dfs[dates[i]].progress_apply(
    lambda row: (
        calculate_similarity(row["qud"], row["question_clean"], pooling_method="max")
        if not pd.isna(row["question"])
        else None
    ),
    axis=1,
)
dfs[dates[i]]["similarity_mean"] = dfs[dates[i]].progress_apply(
    lambda row: (
        calculate_similarity(row["qud"], row["question_clean"], pooling_method="mean")
        if not pd.isna(row["question"])
        else None
    ),
    axis=1,
)
dfs[dates[i]]["similarity_sbert"] = dfs[dates[i]].progress_apply(
    lambda row: (
        calculate_similarity(row["qud"], row["question_clean"], sbert=True)
        if not pd.isna(row["question"])
        else None
    ),
    axis=1,
)

100%|██████████| 144/144 [00:00<00:00, 193.90it/s]


In [136]:
dfs[dates[i]]

,speaker,text,type,election_year,date,candidate,qud,question,similarity_cls,similarity_max,similarity_mean,similarity_sbert,party,question_clean
0,Jim Lehrer,Good evening from the Bushnell Theatre in Hart...,Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
1,Bill Clinton,"Thank you, Jim. And thank you to the people of...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
2,Jim Lehrer,"Senator Dole, two minutes.",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
3,Bob Dole,"Thank you. Thank you, Mr. President, for those...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Republican,NaN
4,Jim Lehrer,"Mr. President, first question. There is a majo...",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Bill Clinton,I think a lot of people deserve credit and Ive...,Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
140,Jim Lehrer,"All right. Thats the last question, the last a...",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN
141,Bill Clinton,"Well, first, Jim, let me thank you and thank y...",Pres,1996,1996-10-06,1,NaN,NaN,NaN,NaN,NaN,NaN,Democratic,NaN
142,Jim Lehrer,"Senator Dole, your closing statement, sir.",Pres,1996,1996-10-06,0,NaN,NaN,NaN,NaN,NaN,NaN,Moderator,NaN


In [137]:
for i in tqdm(range(len(dates))):
    dfs[dates[i]]["similarity_cls"] = dfs[dates[i]].apply(
        lambda row: (
            calculate_similarity(row["qud"], row["question_clean"])
            if not pd.isna(row["question"])
            else None
        ),
        axis=1,
    )
    dfs[dates[i]]["similarity_max"] = dfs[dates[i]].apply(
        lambda row: (
            calculate_similarity(row["qud"], row["question_clean"], pooling_method="max")
            if not pd.isna(row["question"])
            else None
        ),
        axis=1,
    )
    dfs[dates[i]]["similarity_mean"] = dfs[dates[i]].apply(
        lambda row: (
            calculate_similarity(row["qud"], row["question_clean"], pooling_method="mean")
            if not pd.isna(row["question"])
            else None
        ),
        axis=1,
    )
    dfs[dates[i]]["similarity_sbert"] = dfs[dates[i]].apply(
        lambda row: (
            calculate_similarity(row["qud"], row["question_clean"], sbert=True)
            if not pd.isna(row["question"])
            else None
        ),
        axis=1,
    )
    dfs[dates[i]].to_csv(f"{data_base_dir}/{dates[i]}.csv", index=False)

100%|██████████| 13/13 [01:44<00:00,  8.02s/it]
